In [ ]:
import pandas as pd
pd.options.display.max_columns = None
import matplotlib.pyplot as plt
from matplotlib import cm
import seaborn as sns

In [ ]:
datos=[]
ataque=pd.read_csv('data/attacking.csv').drop(['serial','match_played'],axis=1)
intentos=pd.read_csv('data/attempts.csv').drop(['serial','match_played'],axis=1)
defenza=pd.read_csv('data/defending.csv').drop(['serial','match_played'],axis=1)
pases=pd.read_csv('data/distributon.csv').drop(['serial','match_played'],axis=1)
porteros=pd.read_csv('data/goalkeeping.csv').drop(['serial','match_played'],axis=1)
goles=pd.read_csv('data/goals.csv').drop(['serial','match_played'],axis=1)
general=pd.read_csv('data/key_stats.csv').drop(['goals','assists'],axis=1)
datos=[intentos,defenza,pases,porteros,goles,general]
df=ataque
for i in datos:
    df=pd.merge(df,i,on=['player_name','club','position'],how='outer')
df.fillna(0,inplace=True)
df.drop(df[df['player_name']=='Reinildo'].index,inplace=True)
df.drop(df[df['player_name']=='Luis Díaz'].index,inplace=True)
df.drop(df[df['player_name']=='Brobbey'].index,inplace=True)
df['g_m']=df['goals']/df['minutes_played']
df['g_p']=df['goals']/df['match_played']
df['a_m']=df['assists']/df['minutes_played']
df['a_p']=df['assists']/df['match_played']
df.insert(36,'no_penalties',df['goals']-df['penalties'])
equipos=pd.read_csv('data/club_matches.csv', encoding = 'cp1252').drop(['Unnamed: 0'],axis=1).set_index('club')

In [ ]:
df.head()

In [ ]:
df[df['position']=='Goalkeeper'].describe()

In [ ]:
df[df['position']=='Goalkeeper'].describe().iloc[2]

In [ ]:
df[df['position']=='Defender'].describe()

In [ ]:
df[df['position']=='Defender'].describe().iloc[2]

In [ ]:
df[df['position']=='Midfielder'].describe()

In [ ]:
df[df['position']=='Midfielder'].describe().iloc[2]

In [ ]:
df[df['position']=='Forward'].sort_values('goals',ascending=False)

In [ ]:
df[df['position']=='Forward'].describe()

In [ ]:
df

In [ ]:
# Simular seleccion de jugador
jugador='Messi'
df_j=df[df['player_name']==jugador].reset_index(drop=True)
if df_j.shape[0]>1:
    print(df_j[['player_name','club']])
    indice_jugador=int(input(f'Existen {df_j.shape[0]} coincidencias.\nIngresa el numero del jugadar que quieres seleccionar'))
    df_j=df_j.loc[indice_jugador]
df_j

In [ ]:
# Graficar distribucion de goles
df_j.reset_index(drop=True,inplace=True)
if df_j['goals'][0]>0:
    plt.figure(figsize=(14,10))
    colores1=cm.get_cmap('Set1', 4).colors
    colores2=cm.get_cmap('Dark2', 2).colors
    colores3=cm.get_cmap('tab10', 2).colors
    x1=df_j.loc[0,'right_foot':'others'][df_j.loc[0,'right_foot':'others']>0]
    x2=df_j.loc[0,'inside_area':'outside_areas'][df_j.loc[0,'inside_area':'outside_areas']>0]
    x3=df_j.loc[0,'penalties':'no_penalties'][df_j.loc[0,'penalties':'no_penalties']>0]
    x1.plot.pie(legend=True,colors=colores1,wedgeprops={'linewidth':1.8,'edgecolor':"slategray"},autopct='%1.2f%%',textprops={'color':'lavender','fontsize':10},pctdistance=0.88)
    x2.plot.pie(legend=True,colors=colores2,radius=0.75,startangle=90,wedgeprops={'linewidth':1.8,'edgecolor':"slategray"},autopct='%1.2f%%',textprops={'color':'lavender','fontsize':10},pctdistance=0.88)
    x3.plot.pie(legend=True,colors=colores3,radius=0.5,startangle=90,wedgeprops={'linewidth':1.8,'edgecolor':"slategray"},autopct='%1.2f%%',textprops={'color':'lavender','fontsize':10},pctdistance=0.88)
    plt.title(f'Distribucion de goles de {jugador}',fontdict={'fontweight':'bold','color':'w','fontsize':21})
    centre_circle=plt.Circle((0,0),0.25,fc='lightgray')
    fig=plt.gcf()
    fig.gca().add_artist(centre_circle)
    plt.axis('equal')
    plt.legend(loc=7)
    plt.text(-0.1,0.01,f"Goles: {int(df_j['goals'][0])}",fontdict={'fontweight':'bold','fontsize':12})
    plt.text(-0.13,-0.06,f"Partidos: {int(df_j['match_played'][0])}",fontdict={'fontweight':'bold','fontsize':12})
    plt.show()

In [ ]:
# Bigotes por posicion
posicion=df_j['position'][0]
fig,ax=plt.subplots(ncols=6, figsize=(25,10))
df_pos=df[df['position']==posicion]
columnas=['goals','g_p','g_m','assists','a_p','a_m']
for i,col in zip(range(6),columnas):
    ax[i].boxplot(x=df_pos[[col]])
    ax[i].plot(1,df_j[col][0], marker="D", color="red")
    ax[i].tick_params(labelsize=15)
    ax[i].set_xlabel(col,fontsize=20)
plt.tight_layout()

In [ ]:
fig,ax=plt.subplots(ncols=6, figsize=(20,10))
ax[0].boxplot(df_pos['goals'])
ax[0].plot(1,df_j['goals'][0], marker="D", color="red")
# ax[0].annotate(df_j['player_name'][0],(1.1,df_j['goals'][0]))

ax[1].boxplot(df_pos['g_p'])
ax[1].plot(1,df_j['g_p'][0], marker="D", color="red")
# ax[1].annotate(df_j['player_name'][0],(1.1,df_j['g_p'][0]))

In [ ]:
df_j

In [ ]:
df_pos

In [ ]:
# Graficas de todos los jugadores con mas de 3 goles
m=df.sort_values('goals', ascending=False).query("goals>3")
for jugador in m['player_name']:
   df_j=df[df['player_name']==jugador].reset_index(drop=True) 
   plt.figure(figsize=(14,10))
   colores1=cm.get_cmap('Set1', 4).colors
   colores2=cm.get_cmap('Dark2', 2).colors
   colores3=cm.get_cmap('tab10', 2).colors
   x1=df_j.loc[0,'right_foot':'others'][df_j.loc[0,'right_foot':'others']>0]
   x2=df_j.loc[0,'inside_area':'outside_areas'][df_j.loc[0,'inside_area':'outside_areas']>0]
   x3=df_j.loc[0,'penalties':'no_penalties'][df_j.loc[0,'penalties':'no_penalties']>0]
   x1.plot.pie(legend=True,colors=colores1,wedgeprops={'linewidth':1.8,'edgecolor':"slategray"},autopct='%1.2f%%',textprops={'color':'lavender','fontsize':10},pctdistance=0.88)
   x2.plot.pie(legend=True,colors=colores2,radius=0.75,startangle=90,wedgeprops={'linewidth':1.8,'edgecolor':"slategray"},autopct='%1.2f%%',textprops={'color':'lavender','fontsize':10},pctdistance=0.88)
   x3.plot.pie(legend=True,colors=colores3,radius=0.5,startangle=90,wedgeprops={'linewidth':1.8,'edgecolor':"slategray"},autopct='%1.2f%%',textprops={'color':'lavender','fontsize':10},pctdistance=0.88)
   plt.title(f'Distribucion de goles de {jugador}',fontdict={'fontweight':'bold','color':'w','fontsize':21})
   centre_circle=plt.Circle((0,0),0.25,fc='lightgray')
   fig=plt.gcf()
   fig.gca().add_artist(centre_circle)
   plt.axis('equal')
   plt.legend(loc=7)
   plt.text(-0.1,0.01,f"Goles: {int(df_j['goals'][0])}",fontdict={'fontweight':'bold','fontsize':12})
   plt.text(-0.13,-0.06,f"Partidos: {int(df_j['match_played'][0])}",fontdict={'fontweight':'bold','fontsize':12})
   plt.show()